# 1.5 「ファインチューニング」で精度向上を実現する方法

- 本ファイルでは、学習済みのVGGモデルを使用し、ファインチューニングでアリとハチの画像を分類するモデルを学習します



# 学習目標

1.	PyTorchでGPUを使用する実装コードを書けるようになる
2.	最適化手法の設定において、層ごとに異なる学習率を設定したファインチューニングを実装できるようになる
3.	学習したネットワークを保存・ロードできるようになる



# 事前準備

- 1.4節で解説したAWS EC2 のGPUインスタンスを使用します


In [ ]:
# パッケージのimport
import random
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm

In [ ]:
print(torch.__version__, torch.cuda.is_available())

In [ ]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [ ]:
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

batch_size = 32

In [ ]:
OUTPUT_FEATURES = 2

MODEL_CATEGORY = 'VGG'

OPTIMIZER_TYPE = 'Adam'
LEARNING_RATIO = 0.001
MOMENTUM = 0.0
WEIGHT_DECAY = 1e-6

NUM_EPOCHS = 30

# MLflow

In [ ]:
import mlflow

In [ ]:
# experimentの作成(読み込み)
experiment_id = mlflow.set_experiment(MODEL_CATEGORY)  # experimentの設定. 無ければ新規に作成.
print(experiment_id.experiment_id)

# DatasetとDataLoaderを作成

In [ ]:
# 1.3節で作成したクラスを同じフォルダにあるmake_dataset_dataloader.pyに記載して使用
from utils.dataloader_image_classification import (
    HymenopteraDataset,
    ImageTransform,
    make_datapath_list,
)

# アリとハチの画像へのファイルパスのリストを作成する
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

# Datasetを作成する
train_dataset = HymenopteraDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase="train"
)

val_dataset = HymenopteraDataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase="val"
)


# DataLoaderを作成する
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False
)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

# モデルを学習させる関数を作成

In [ ]:
def plot_history(name, history):
    fig, axes = plt.subplots(1, 2, figsize=(10, 4), tight_layout=True)
    axes[0].plot(
        range(len(history["train"]["loss"])),
        history["train"]["loss"],
        "r-o",
        label=f"{name}-train",
    )
    axes[0].plot(
        range(len(history["val"]["loss"])),
        history["val"]["loss"],
        "b--s",
        label=f"{name}-val",
    )
    axes[0].set_xlabel("Epochs", size=14)
    axes[0].set_ylabel("Loss", size=14)
    axes[0].tick_params(labelsize=12)
    axes[0].grid()
    axes[0].legend()

    axes[1].plot(
        range(len(history["train"]["acc"])),
        history["train"]["acc"],
        "r-o",
        label=f"{name}-train",
    )
    axes[1].plot(
        range(len(history["val"]["acc"])),
        history["val"]["acc"],
        "b--s",
        label=f"{name}-val",
    )
    axes[1].set_xlabel("Epochs", size=14)
    axes[1].set_ylabel("Accuracy", size=14)
    axes[1].tick_params(labelsize=12)
    axes[1].grid()
    axes[1].legend()
    plt.suptitle(f"{name}", size=16)
    # plt.show()
    return fig

In [ ]:
def train_model(name, net, dataloaders_dict, criterion, optimizer, num_epochs):
    # 初期設定
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    history = {}
    history["train"] = {}
    history["val"] = {}
    history["train"]["loss"] = []
    history["train"]["acc"] = []
    history["val"]["loss"] = []
    history["val"]["acc"] = []

    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch + 1, num_epochs))
        # print("-------------")

        # epochごとの訓練と検証のループ
        for phase in ["train", "val"]:
            if phase == "train":
                net.train()  # モデルを訓練モードに
            else:
                net.eval()  # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
            # if (epoch == 0) and (phase == "train"):
            #     continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                # GPUが使えるならGPUにデータを送る
                inputs = inputs.to(device)
                labels = labels.to(device)

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == "train"):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.cpu().double() / len(dataloaders_dict[phase].dataset)

            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            history[phase]["loss"].append(epoch_loss)
            history[phase]["acc"].append(epoch_acc)

            # MLflow: track metrics
            mlflow.log_metric(f"{phase}_loss", epoch_loss, step=epoch)  # Train Loss
            mlflow.log_metric(f"{phase}_acc", epoch_acc, step=epoch)  # Train Loss

    return history

# 損失関数を定義

In [ ]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()

# ネットワークモデルの作成

In [ ]:
def replace_last_layer_vgg(_net, output_features):
    last_in_features = _net.classifier[-1].in_features
    _net.classifier[-1] = nn.Linear(in_features=last_in_features, out_features=output_features)
    return _net

def params_in_last_layers(_net):
    params_to_update = []
    for param in _net.classifier[-1].parameters():
        param.requires_grad = True
        params_to_update.append(param)
    return params_to_update

In [ ]:
vgg_models = {
    "vgg16": {'model': models.vgg16, 'weights': models.VGG16_Weights.DEFAULT, 'layers': 16},
    "vgg16_bn": {'model': models.vgg16_bn, 'weights': models.VGG16_BN_Weights.DEFAULT, 'layers': 17},
    "vgg19": {'model': models.vgg19, 'weights': models.VGG19_Weights.DEFAULT, 'layers': 19},
    "vgg19_bn": {'model': models.vgg19_bn, 'weights': models.VGG19_BN_Weights.DEFAULT, 'layers': 20},
}

In [ ]:
mlflow.end_run()

# Start run

In [ ]:
save_dir = '../../../model'

In [ ]:
vgg_results = {}
for name, model_dict in vgg_models.items():
    model = model_dict["model"]
    weights = model_dict["weights"]

    # MLflow: runの作成
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = f"{name}_{OPTIMIZER_TYPE}_LR{LEARNING_RATIO}_EPOCH{NUM_EPOCHS}_{now}"
    mlflow_run = mlflow.start_run(
        experiment_id=experiment_id.experiment_id,  # set_experimentの返り値を入れる.
        run_name=run_name,  # run_nameに、作成時刻を用いるようにした.
    )

    # MLflow: track params
    mlflow.log_params(
        {
            "Model": name,
            "Layers": model_dict["layers"],
            "Learning_ratio": LEARNING_RATIO,  # Learning ratio
            "Num_epochs": NUM_EPOCHS,  # num of Epochs
            "Optimizer": OPTIMIZER_TYPE,  # optimizer
        }
    )

    _net = model(weights=weights)
    _net = replace_last_layer_vgg(_net, OUTPUT_FEATURES)
    _net.train()
    print("Model:", name)

    # 学習させるパラメータ以外は勾配計算をなくし、変化しないように設定
    for param in _net.parameters():
        param.requires_grad = False

    params_to_update = params_in_last_layers(_net)

    if OPTIMIZER_TYPE=='SGD':
        optimizer = optim.SGD(
            params_to_update,
            lr=LEARNING_RATIO,
            momentum=MOMENTUM,
        )
        mlflow.log_params(
            {
                "Momentum": MOMENTUM,
            }
        )
    elif OPTIMIZER_TYPE=='Adam':
        optimizer = optim.Adam(
            params_to_update,
            lr=LEARNING_RATIO,
            weight_decay=WEIGHT_DECAY,
        )
        mlflow.log_params(
            {
                "Weight_decay": WEIGHT_DECAY,
            }
        )

    history = train_model(
        name, _net, dataloaders_dict, criterion, optimizer, num_epochs=NUM_EPOCHS
    )
    vgg_results[name] = history

    # PyTorchのネットワークパラメータの保存
    save_name = f"{name}_weights_fine_tuning.pth"
    save_path = os.path.join(save_dir, save_name)
    torch.save(_net.state_dict(), save_path)
    
    mlflow.log_artifact(save_path)

    # # MLflow: track figure
    # figure = plot_history(name, history)
    # mlflow.log_figure(figure, "history.png")

    # MLflowL end run
    mlflow.end_run()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4), tight_layout=True)
for model in vgg_results.keys():
    # for phase in vgg_results[model].keys():
    for phase in ["val"]:
        axes[0].plot(
            range(len(vgg_results[model][phase]["loss"])),
            vgg_results[model][phase]["loss"],
            # plot_style[phase],
            label=f"{model}-{phase}",
        )
axes[0].set_xlabel("Epochs", size=14)
axes[0].set_ylabel("Loss", size=14)
axes[0].tick_params(labelsize=12)
axes[0].grid()
axes[0].legend()

for model in vgg_results.keys():
    # for phase in vgg_results[model].keys():
    for phase in ["val"]:
        axes[1].plot(
            range(len(vgg_results[model][phase]["acc"])),
            vgg_results[model][phase]["acc"],
            # plot_style[phase],
            label=f"{model}-{phase}",
        )
axes[1].set_xlabel("Epochs", size=14)
axes[1].set_ylabel("Accuracy", size=14)
axes[1].tick_params(labelsize=12)
axes[1].grid()
axes[1].legend()
plt.show()

# 学習したネットワークを保存・ロード

In [ ]:
# # PyTorchのネットワークパラメータの保存
# save_path = "./weights_fine_tuning.pth"
# torch.save(net.state_dict(), save_path)

In [ ]:
# # PyTorchのネットワークパラメータのロード
# load_path = "./weights_fine_tuning.pth"
# load_weights = torch.load(load_path)
# net.load_state_dict(load_weights)

# # GPU上で保存された重みをCPU上でロードする場合
# load_weights = torch.load(load_path, map_location={"cuda:0": "cpu"})
# net.load_state_dict(load_weights)

以上